In [45]:
# importing necessary libraries
import pandas as pd  
import numpy as np
import ast
from sklearn.preprocessing import MinMaxScaler

In [46]:
df=pd.read_csv('TRADES_CopyTr_90D_ROI.csv')

In [47]:
df=pd.DataFrame(df)

In [48]:
df

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."
...,...,...
145,4000222729738650369,"[{'time': 1718982068000, 'symbol': 'ARKMUSDT',..."
146,3998659472131949824,"[{'time': 1718979385000, 'symbol': 'BTCUSDT', ..."
147,4028701921959171840,"[{'time': 1718984241000, 'symbol': 'BTCUSDT', ..."
148,4014818740371615232,"[{'time': 1718983357000, 'symbol': 'SOLUSDT', ..."


In [49]:
trade_history_list = []

max_iterations = min(150, len(df))  

for i in range(max_iterations):
    port_id = df['Port_IDs'].iloc[i]
    
    trade_history = df.loc[df['Port_IDs'] == port_id, 'Trade_History'].values
    
    if len(trade_history) == 0 or pd.isna(trade_history[0]):
        print(f"Trade history for Port_ID {port_id} is NaN or not found, skipped it.")
        continue
    
    try:
        trade_history_parsed = ast.literal_eval(trade_history[0])
        
        trade_history_df = pd.json_normalize(trade_history_parsed)
        
        trade_history_df['Port_ID'] = port_id
        
        trade_history_list.append(trade_history_df)

    except (ValueError, SyntaxError) as e:
        print(f"Error parsing Trade_History for Port_ID {port_id}: {e}")

if trade_history_list:  
    combined_trade_history_df = pd.concat(trade_history_list, ignore_index=True)
    
    cols = ['Port_ID'] + [col for col in combined_trade_history_df.columns if col != 'Port_ID']
    combined_trade_history_df = combined_trade_history_df[cols]
    
else:
    print("No valid trade histories found.")



Trade history for Port_ID 3919965573764932864 is NaN or not found, skipped it.


In [50]:
combined_trade_history_df

,Port_ID,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy
0,3925368433214965504,1718899656000,SOLUSDT,BUY,132.537000,-0.994027,USDT,1988.055000,USDT,0.0,USDT,SOL,15.0,LONG,True
1,3925368433214965504,1718899618000,DOGEUSDT,BUY,0.121820,-0.279796,USDT,1398.980880,USDT,0.0,USDT,DOGE,11484.0,LONG,False
2,3925368433214965504,1718899618000,DOGEUSDT,BUY,0.121820,-0.039494,USDT,197.470220,USDT,0.0,USDT,DOGE,1621.0,LONG,False
3,3925368433214965504,1718899616000,DOGEUSDT,BUY,0.121820,-0.008284,USDT,16.567520,USDT,0.0,USDT,DOGE,136.0,LONG,True
4,3925368433214965504,1718899616000,DOGEUSDT,BUY,0.121820,-0.046109,USDT,92.217740,USDT,0.0,USDT,DOGE,757.0,LONG,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211272,3768170840939476993,1718549875000,JASMYUSDT,SELL,0.035968,-0.025070,USDT,50.139392,USDT,0.0,USDT,JASMY,1394.0,BOTH,False
211273,3768170840939476993,1718549875000,JASMYUSDT,SELL,0.035968,-0.008255,USDT,16.509312,USDT,0.0,USDT,JASMY,459.0,BOTH,False
211274,3768170840939476993,1718549875000,JASMYUSDT,SELL,0.035969,-0.031545,USDT,63.089626,USDT,0.0,USDT,JASMY,1754.0,BOTH,False
211275,3768170840939476993,1718549875000,JASMYUSDT,SELL,0.035969,-0.031905,USDT,63.809006,USDT,0.0,USDT,JASMY,1774.0,BOTH,False


In [86]:
combined_trade_history_df.to_csv('new.csv')

In [87]:
df=pd.read_csv('new.csv')
df.duplicated().sum()

0

In [88]:
df.columns

Index(['Unnamed: 0', 'Port_ID', 'time', 'symbol', 'side', 'price', 'fee',
       'feeAsset', 'quantity', 'quantityAsset', 'realizedProfit',
       'realizedProfitAsset', 'baseAsset', 'qty', 'positionSide', 'activeBuy'],
      dtype='object')

In [89]:
df_cat=df[['symbol','feeAsset','quantityAsset','realizedProfitAsset','baseAsset','positionSide','activeBuy']]
df_cat

,symbol,feeAsset,quantityAsset,realizedProfitAsset,baseAsset,positionSide,activeBuy
0,SOLUSDT,USDT,USDT,USDT,SOL,LONG,True
1,DOGEUSDT,USDT,USDT,USDT,DOGE,LONG,False
2,DOGEUSDT,USDT,USDT,USDT,DOGE,LONG,False
3,DOGEUSDT,USDT,USDT,USDT,DOGE,LONG,True
4,DOGEUSDT,USDT,USDT,USDT,DOGE,LONG,True
...,...,...,...,...,...,...,...
211272,JASMYUSDT,USDT,USDT,USDT,JASMY,BOTH,False
211273,JASMYUSDT,USDT,USDT,USDT,JASMY,BOTH,False
211274,JASMYUSDT,USDT,USDT,USDT,JASMY,BOTH,False
211275,JASMYUSDT,USDT,USDT,USDT,JASMY,BOTH,False


In [90]:
df_cat['positionSide'].unique()

array(['LONG', 'SHORT', 'BOTH'], dtype=object)

In [91]:
df['side'].unique()

array(['BUY', 'SELL'], dtype=object)

In [92]:
df_regr=df.drop(columns=df_cat,axis=0)
df_regr

,Unnamed: 0,Port_ID,time,side,price,fee,quantity,realizedProfit,qty
0,0,3925368433214965504,1718899656000,BUY,132.537000,-0.994027,1988.055000,0.0,15.0
1,1,3925368433214965504,1718899618000,BUY,0.121820,-0.279796,1398.980880,0.0,11484.0
2,2,3925368433214965504,1718899618000,BUY,0.121820,-0.039494,197.470220,0.0,1621.0
3,3,3925368433214965504,1718899616000,BUY,0.121820,-0.008284,16.567520,0.0,136.0
4,4,3925368433214965504,1718899616000,BUY,0.121820,-0.046109,92.217740,0.0,757.0
...,...,...,...,...,...,...,...,...,...
211272,211272,3768170840939476993,1718549875000,SELL,0.035968,-0.025070,50.139392,0.0,1394.0
211273,211273,3768170840939476993,1718549875000,SELL,0.035968,-0.008255,16.509312,0.0,459.0
211274,211274,3768170840939476993,1718549875000,SELL,0.035969,-0.031545,63.089626,0.0,1754.0
211275,211275,3768170840939476993,1718549875000,SELL,0.035969,-0.031905,63.809006,0.0,1774.0


In [93]:
df_regr['side'] = df_regr['side'].replace({'BUY':0,'SELL':1}) # using replace() for label encoding

In [94]:
df_regr.columns

Index(['Unnamed: 0', 'Port_ID', 'time', 'side', 'price', 'fee', 'quantity',
       'realizedProfit', 'qty'],
      dtype='object')

In [95]:
df_regr.drop(columns=['Unnamed: 0'],axis=0)

,Port_ID,time,side,price,fee,quantity,realizedProfit,qty
0,3925368433214965504,1718899656000,0,132.537000,-0.994027,1988.055000,0.0,15.0
1,3925368433214965504,1718899618000,0,0.121820,-0.279796,1398.980880,0.0,11484.0
2,3925368433214965504,1718899618000,0,0.121820,-0.039494,197.470220,0.0,1621.0
3,3925368433214965504,1718899616000,0,0.121820,-0.008284,16.567520,0.0,136.0
4,3925368433214965504,1718899616000,0,0.121820,-0.046109,92.217740,0.0,757.0
...,...,...,...,...,...,...,...,...
211272,3768170840939476993,1718549875000,1,0.035968,-0.025070,50.139392,0.0,1394.0
211273,3768170840939476993,1718549875000,1,0.035968,-0.008255,16.509312,0.0,459.0
211274,3768170840939476993,1718549875000,1,0.035969,-0.031545,63.089626,0.0,1754.0
211275,3768170840939476993,1718549875000,1,0.035969,-0.031905,63.809006,0.0,1774.0


In [96]:
unique_port_ids=df_regr['Port_ID'].unique()
unique_port_ids[0]

3925368433214965504

In [97]:
metrics_df = pd.DataFrame()

In [98]:
metrics_df['portid'] = unique_port_ids 

In [99]:
def calc_pnl(df_regr,port_id):
    port_df = df_regr[df_regr['Port_ID'] == port_id]
    return port_df['realizedProfit'].sum()

In [100]:
pnl_values=[]

In [101]:
for port_id in metrics_df['portid']:
    pnl = calc_pnl(df_regr, port_id)
    pnl_values.append(pnl)


In [102]:
metrics_df['pnl']=pnl_values

In [103]:
metrics_df

,portid,pnl
0,3925368433214965504,6789.436739
1,4002413037164645377,3686.969761
2,3923766029921022977,772.729032
3,3994879592543698688,3658.765062
4,3926423286576838657,1309.459960
...,...,...
144,4000222729738650369,1803.620133
145,3998659472131949824,1449.325940
146,4028701921959171840,17601.401398
147,4014818740371615232,2013.647510


In [105]:
def calculate_investment(df_regr, port_id):
    port_df = df_regr[df_regr['Port_ID'] == port_id]
    buy_trades = port_df[port_df['side'] == 0]  # 'BUY' is replaced to 0
    total_investment = (buy_trades['price'] * buy_trades['qty']).sum()
    return total_investment

def calculate_roi(total_pnl, total_investment):
    if total_investment == 0:
        return 0  
    return (total_pnl / total_investment) * 100

In [106]:
def calculate_sharpe_ratio(df_regr, port_id):
    port_df = df_regr[df_regr['Port_ID'] == port_id]
    daily_returns = port_df['realizedProfit']
    mean_return = daily_returns.mean()
    std_return = daily_returns.std()
    if std_return == 0:
        return 0
    return mean_return / std_return

In [107]:
def calculate_mdd(df_regr, port_id):
    port_df = df_regr[df_regr['Port_ID'] == port_id]
    cum_returns = port_df['realizedProfit'].cumsum()
    max_value = cum_returns.cummax()
    drawdown = (cum_returns - max_value) / max_value
    return drawdown.min()

In [108]:
def calculate_win_rate(df_regr, port_id):
    port_df = df_regr[df_regr['Port_ID'] == port_id]
    total_positions = len(port_df)
    win_positions = len(port_df[port_df['realizedProfit'] > 0])
    win_rate = (win_positions / total_positions) * 100 if total_positions > 0 else 0
    return win_rate, win_positions, total_positions

In [109]:
roi_values = []
sharpe_values = []
mdd_values = []
win_rate_values = []
win_positions_values = []
total_positions_values = []

In [110]:
for port_id in metrics_df['portid']:
    pnl = calc_pnl(df_regr, port_id)
    total_investment = calculate_investment(df_regr, port_id)
    roi = calculate_roi(pnl, total_investment)
    sharpe_ratio = calculate_sharpe_ratio(df_regr, port_id)
    mdd = calculate_mdd(df_regr, port_id)
    win_rate, win_positions, total_positions = calculate_win_rate(df_regr, port_id)
    
    pnl_values.append(pnl)
    roi_values.append(roi)
    sharpe_values.append(sharpe_ratio)
    mdd_values.append(mdd)
    win_rate_values.append(win_rate)
    win_positions_values.append(win_positions)
    total_positions_values.append(total_positions)

In [111]:
metrics_df['ROI'] = roi_values
metrics_df['Sharpe_Ratio'] = sharpe_values
metrics_df['MDD'] = mdd_values
metrics_df['Win_Rate'] = win_rate_values
metrics_df['Win_Positions'] = win_positions_values
metrics_df['Total_Positions'] = total_positions_values

In [112]:
metrics_df

,portid,pnl,ROI,Sharpe_Ratio,MDD,Win_Rate,Win_Positions,Total_Positions
0,3925368433214965504,6789.436739,0.735604,0.274873,-0.040337,33.333333,486,1458
1,4002413037164645377,3686.969761,4.965881,0.125783,-0.046572,18.576389,107,576
2,3923766029921022977,772.729032,0.313468,0.072381,-0.354619,28.514694,359,1259
3,3994879592543698688,3658.765062,1.670531,0.119384,-0.156883,55.283308,361,653
4,3926423286576838657,1309.459960,0.271285,0.050260,-0.542381,34.100418,163,478
...,...,...,...,...,...,...,...,...
144,4000222729738650369,1803.620133,1.470158,0.248086,-0.610725,56.054280,537,958
145,3998659472131949824,1449.325940,0.616699,0.382438,0.000000,48.979592,72,147
146,4028701921959171840,17601.401398,0.798593,0.275724,-0.045177,54.294032,373,687
147,4014818740371615232,2013.647510,0.458144,0.191113,-0.739679,20.982987,111,529


In [113]:
metrics_df['MDD'] = metrics_df['MDD'].fillna(metrics_df['MDD'].mean())


In [114]:
metrics_df['MDD'] = metrics_df['MDD'].replace([-np.inf], np.nan)  
metrics_df['MDD'] = metrics_df['MDD'].fillna(metrics_df['MDD'].mean()) 
metrics_df['MDD'] = pd.to_numeric(metrics_df['MDD'], errors='coerce') 
metrics_df['MDD'] = metrics_df['MDD'].fillna(0) 

In [116]:
scaler = MinMaxScaler()

normalized_metrics = scaler.fit_transform(metrics_df[['pnl', 'ROI', 'Sharpe_Ratio', 'Win_Rate', 'Total_Positions']])
normalized_df = pd.DataFrame(normalized_metrics, columns=['pnl', 'ROI', 'Sharpe_Ratio', 'Win_Rate', 'Total_Positions'])

mdd_scaled = scaler.fit_transform(metrics_df[['MDD']])
normalized_df['MDD'] = 1 - mdd_scaled.flatten()

In [117]:
final_metrics = pd.concat([metrics_df[['portid']], normalized_df], axis=1)


In [122]:

weights = {
    'pnl': 0.40,              
    'ROI': 0.30,              
    'Sharpe_Ratio': 0.15,     
    'Win_Rate': 0.10,         
    'Total_Positions': 0.05    
}



final_metrics['Score'] = (final_metrics['pnl'] * weights['pnl'] +
                           final_metrics['ROI'] * weights['ROI'] +
                           final_metrics['Sharpe_Ratio'] * weights['Sharpe_Ratio'] +
                           final_metrics['Win_Rate'] * weights['Win_Rate'] +
                           final_metrics['Total_Positions'] * weights['Total_Positions'])



In [128]:
final_metrics['Rank'] = final_metrics['Score'].rank(ascending=False)

first_20_accounts = final_metrics.sort_values(by='Rank').head(20)

first_20_accounts

,portid,pnl,ROI,Sharpe_Ratio,Win_Rate,Total_Positions,MDD,Score,Rank
82,4020204877254599680,1.000000,0.031264,0.184159,0.410437,0.994235,4.426222e-02,0.527758,1.0
148,3768170840939476993,0.149271,1.000000,0.666914,0.533971,0.000000,0.000000e+00,0.513143,2.0
130,3826087012661391104,0.152697,0.557811,1.000000,0.726793,0.015483,2.775792e-04,0.451876,3.0
126,3999240873283311617,0.651145,0.045559,0.405607,0.651896,0.742547,2.911892e-04,0.437284,4.0
35,3986814617275053313,0.340079,0.063991,0.541433,0.974589,0.583100,8.209669e-07,0.363058,5.0
118,3907081197088384000,0.359980,0.079710,0.399265,0.764966,0.679130,1.339210e-03,0.338248,6.0
85,4022641794255717633,0.172191,0.123348,0.845473,0.542264,0.994564,3.084566e-03,0.336656,7.0
140,4039129759104249600,0.161372,0.214673,0.796743,0.552706,0.019601,0.000000e+00,0.304713,8.0
128,3956048468100538880,0.162667,0.095263,0.800487,0.889951,0.002306,0.000000e+00,0.302829,9.0
21,3891020560590657281,0.180246,0.133116,0.677803,0.806862,0.069676,0.000000e+00,0.297874,10.0


In [129]:
first_20_scores=first_20_accounts[['portid','Score']]

In [130]:
first_20_scores.to_csv('top20.csv')

In [131]:
first_20_accounts.to_csv('top20accounts.csv')